
# Homework 2: Portfolio Optimization



## Question 1

In [8]:
# Import gurobi and numpy
from gurobipy import *
import numpy as np
from numpy import genfromtxt
import csv

## Get index of 4 tickers
tick4 = ["MSFT","GS","PG","SCHP"];

# Get variable names
with open('Prices.csv') as csvFile:
    reader = csv.reader(csvFile)
    tickers = next(reader) ## stores the tickets of all 390 stocks

tickind =[];
for t in tick4:
    tickind.append(tickers.index(t)) ## collect index that corresponds to each ticker

# Load data
prices = genfromtxt('Prices.csv', delimiter=',',skip_header = 1)

# get dimensions of data
d = prices.shape[0]
n = prices.shape[1]

# calculate monthly returns of each stock
returns = np.zeros((d-1,n))
for stock in range(n):
    for month in range(d-1):
        returns[month,stock] = prices[month+1,stock]/prices[month,stock]-1
        
# Store average return (parameter r_i in portfolio optimization model)       
#avg_return = np.zeros(n)
avg_return = np.mean(returns,axis=0)

# Store covariance matrix (parameter C_ij in portfolio optimization model)
C = np.zeros((n,n))
C = np.cov(np.transpose(returns))


In [9]:
type(avg_return)

numpy.ndarray

### Model 1

In [19]:
mod = Model()
n = 4

## ADDING DECISION VARIABLE 
x = {}
for i in tickind:
    x[i] = mod.addVar(lb = 0, vtype = GRB.CONTINUOUS)

## ADDING CONSTRAINTS

mod.addConstr(sum(x[i] for i in tickind) == 1) ## Sum of weights = 1

mod.addConstr(sum(avg_return[i]*x[i] for i in tickind) >= 0.005) ## Portfolio return >= 0.5

## ADDING OBJECTIVE 

mod.setObjective(sum(sum(x[i]*x[j] * C[i,j] for j in tickind) for i in tickind),GRB.MINIMIZE) ## Minimize Variance of Portfolio to minimize risk


In [20]:
mod.update()
mod.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 2 rows, 4 columns and 8 nonzeros
Model fingerprint: 0xcd87a4af
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-05, 7e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve time: 0.00s
Presolved: 2 rows, 4 columns, 8 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.87864836e+05 -1.87864836e+05  3.50e+03 1.91e-07  1.00e+06     0s
   1   5.03195868e+03 -5.16248579e+03  2.59e+02 1.41e-08

In [37]:
   
stocks = []
for v in mod.getVars():
    stocks.append(v.x)
print('Optimal Risk Value:',mod.ObjVal,'\n')
print('Model Solver Time: 0.02 seconds','\n')
print('Weights of stocks:')
print('   MSFT:',stocks[0]*100,'%\n','   GS:',stocks[1]*100,'%\n','   PG:',stocks[2]*100,'%\n','   SCHP:',stocks[3]*100,'%\n')

Optimal Risk Value: 0.0001774932651657724 

Model Solver Time: 0.02 seconds 

Weights of stocks:
   MSFT: 23.711749137102103 %
    GS: 2.585983533068454 %
    PG: 1.5749710132180152e-07 %
    SCHP: 73.70226717233427 %



### Model 2

In [17]:

mod2 = Model()
n = 390
## ADDING DECISION VARIABLE 

x = mod2.addVars(390)

## ADDING CONSTRAINTS
for i in range(n):
    mod2.addConstr(x[i]>=0)

mod2.addConstr(sum(x[i] for i in range(n)) == 1) ## Sum of weights = 100%

mod2.addConstr(sum(avg_return[i]*x[i] for i in range(n)) >= 0.005) ## Portfolio return >= 0.5

## ADDING OBJECTIVE 

mod2.setObjective(sum(sum(x[i]*x[j] * C[i,j] for j in range(n)) for i in range(n)),GRB.MINIMIZE) ## Minimize Variance of Portfolio to minimize risk



In [18]:
mod2.update()
mod2.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 392 rows, 390 columns and 1170 nonzeros
Model fingerprint: 0x117c1e4d
Model has 76245 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 390 columns, 780 nonzeros
Presolved model has 76245 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 59
 AA' NZ     : 1.830e+03
 Factor NZ  : 1.891e+03
 Factor Ops : 7.753e+04 (less than 1 second per iteration)
 Threads    : 8

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.89821559e-13 -2.89821559e-13  3.90e+05 3.22e-07  1.00e+06     0

In [38]:
print('Optimal Risk Value:',mod2.ObjVal,'\n')
print('Model Solver Time: 0.05 seconds','\n')

Optimal Risk Value: 2.8785756399295463e-05 

Model Solver Time: 0.05 seconds 



### Model 3

In [104]:
mod3 = Model()
n = 390

## ADDING DECISION VARIABLE 

x = mod3.addVars(390)
y = mod3.addVars(390,vtype = GRB.BINARY)


## ADDING CONSTRAINTS
for i in range(n):
    mod3.addConstr(x[i]>=0)
    mod3.addConstr(y[i]>=x[i])
    
mod3.addConstr(sum(x[i]*y[i] for i in range(n)) == 1) ## Sum of weights = 1
mod3.addConstr(sum(y[i] for i in range(n)) <= 4)
mod3.addConstr(sum(avg_return[i]*x[i] for i in range(n)) >= 0.005) ## Portfolio return >= 0.5%



## ADDING OBJECTIVE 

mod3.setObjective(sum(sum(x[i]*x[j]*C[i,j] for j in range(n)) for i in range(n)),GRB.MINIMIZE) ## Minimize Variance of Portfolio to minimize risk


In [105]:
mod3.update()
mod3.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 782 rows, 780 columns and 1950 nonzeros
Model fingerprint: 0x4cd4185c
Model has 76245 quadratic objective terms
Model has 1 quadratic constraint
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
  QRHS range       [1e+00, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 1563 rows, 1170 columns, 4680 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 780 continuous, 390 integer (390 binary)
Found heuristic solution: objective 0.0009032

Root relaxation: objective 2.878501e-05, 9116 iterations, 0.57 seconds

    Nodes    |    Current Node    |     Ob

In [97]:
stocks = []
for v in mod3.getVars():
    stocks.append(v.x)
print('Optimal Risk Value:',mod3.ObjVal,'\n')
print('Model Solver Time: 41.68 seconds','\n')
print('Weights of stocks:')
for x in stocks: 
    if x != 0 and x!= 1:
        print(tickers[stocks.index(x)],':',x*100,'%')
        

Optimal Risk Value: 6.753470760728254e-05 

Model Solver Time: 40.78 seconds 

Weights of stocks:
CME : 12.641141929490557 %
LLY : 7.547619035437594 %
NVDA : 4.375370672843286 %
BND : 75.43586836222858 %


__________

## Question 2

 a) The optimal risk value in model 2 is less than that of model 1. This is because we have more constraints in model 1. (Only allowed to choose 4 known stocks)
 
 b) The optimal risk value in model 2 is less than that of moddel 3. This is because we have more constraints in model 3. ( Only allowed to choose 4 unknown stocks) 

_______

## Question 3

### Part A: 

In [99]:
mod3.Params.TimeLimit = 30.0
mod3.update()
mod3.optimize()

Changed value of parameter TimeLimit to 30.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 782 rows, 780 columns and 1950 nonzeros
Model fingerprint: 0x4cd4185c
Model has 76245 quadratic objective terms
Model has 1 quadratic constraint
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
  QRHS range       [1e+00, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.02s
Presolved: 1563 rows, 1170 columns, 4680 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 780 continuous, 390 integer (390 binary)
Found heuristic solution: objective 0.0009032

Root relaxation: objec

The value is almost the same as the one achieved in Q1.C

In [100]:
print('Optimal value for Q1C: 6.753470760728254e-05')
print('Optimal value for Q3A: 6.078512891328e-05')

Optimal value for Q1C: 6.753470760728254e-05
Optimal value for Q3A: 6.078512891328e-05


### Part B: 

In [106]:
mod3.Params.MIPGap = 0.1
mod3.update()
mod3.optimize()

Changed value of parameter MIPGap to 0.1
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 782 rows, 780 columns and 1950 nonzeros
Model fingerprint: 0x4cd4185c
Model has 76245 quadratic objective terms
Model has 1 quadratic constraint
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
  QRHS range       [1e+00, 1e+00]
Presolved: 1563 rows, 1170 columns, 4680 nonzeros
Presolved model has 76245 quadratic objective terms

Continuing optimization...


Cutting planes:
  Cover: 1
  MIR: 19
  Flow cover: 10

Explored 90481 nodes (3698456 simplex iterations) in 0.01 seconds
Thread count was 16 (of 16 availab

The model was solved a lot faster than in Q1.C 

In [107]:
print('Q1C Solver Time: 40.78 seconds')
print('Q3B Solver Time: 0.01 seconds')

Q1C Solver Time: 40.78 seconds
Q3B Solver Time: 0.01 seconds


It seems that modifying the gap in this case would be more effective than setting a time limit. 

___________